In [ ]:
import numpy as np
import pandas as pd
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

import spacy
from nltk.corpus import stopwords

In [2]:
english_stopwords = stopwords.words("english")
data = pd.read_csv(r"C:\Users\alkrd\Desktop\graduation_project\the_project\preprocssed_data\cleaned_data_USA.csv")
dataFrame = data[['newsSnippet']]

In [3]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    print("lemmatizing...")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

lemmatized_texts = lemmatization(dataFrame['newsSnippet'].astype(str).tolist())
print("lemmitized sample: ", lemmatized_texts[100])

lemmatizing...
lemmitized sample:  championship game feature accord most ranking good team result flash brilliance sloppy play defeat night


In [4]:
def gen_words(texts):
    print("removing stop words...")
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print("after removing stop words: ",data_words[100])

removing stop words...
after removing stop words:  ['championship', 'game', 'feature', 'accord', 'most', 'ranking', 'good', 'team', 'result', 'flash', 'brilliance', 'sloppy', 'play', 'defeat', 'night']


In [5]:
id2word = corpora.Dictionary(data_words)
corpus = []
print("doing some shi...")
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
print("corpus? ", corpus[0])

doing some shi...
corpus?  [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1)]


In [6]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=30,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)

In [ ]:




pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
